In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

account_data_01 = pd.read_csv('results/dataset_analysis.csv', low_memory = False)
clusters = pd.read_csv('results/clusters.csv', low_memory = False)
account_data_02 = account_data_01.merge(clusters, on = 'image_id', how = 'inner')

print('Number of samples in account_data_01:', account_data_01.shape[0])
print('Number of samples in account_data_02:', account_data_02.shape[0])

In [ ]:
# let's create a feature matrix, a cluster vector and a target variable
X = account_data_02.drop(['likes_groups', 'log_likes_image_corrected', 'likes_image_corrected', 'cluster', 'image_id', 'resort', 'accountname'], axis = 1)
clusters = account_data_02['cluster']
image_ids = account_data_02['image_id']
Y = account_data_02['likes_groups'].apply(lambda x: int(str(x)[0]))

stratification = pd.concat([Y, clusters], axis = 1)

In [ ]:
X.shape, stratification.shape, Y.shape, stratification.shape

In [ ]:
# split the data into a training-, validation and test set

X_temp, X_test, \
Y_temp, Y_test, \
image_ids_temp, image_ids_test, \
stratification_temp, stratification_test = train_test_split(X,
                                                            Y,
                                                            image_ids,
                                                            stratification,
                                                            stratify = stratification,
                                                            test_size = 0.2,
                                                            train_size = 0.8,
                                                            random_state = 0)

X_train, X_val, \
Y_train, Y_val, \
image_ids_train, image_ids_val, \
stratification_train, stratification_val = train_test_split(X_temp,
                                                            Y_temp,
                                                            image_ids_temp,
                                                            stratification_temp,
                                                            stratify = stratification_temp,
                                                            test_size = 0.25,
                                                            train_size = 0.75,
                                                            random_state = 0)

# print the number of examples in and dimensions of each set

print ('Number of training examples:', X_train.shape[0])
print ('Number of validation examples:', X_val.shape[0])
print ('Number of test examples:', X_test.shape[0])

print ('X_train shape:', X_train.shape)
print ('X_valshape:', X_val.shape)
print ('X_test shape:', X_test.shape)

print ('Y_train shape:', Y_train.shape)
print ('Y_val shape:', Y_val.shape)
print ('Y_test shape:', Y_test.shape)

# check the stratification

clusters_training_perc = stratification_train['cluster'].sum() / stratification_train.shape[0]
clusters_validation_perc = stratification_val['cluster'].sum() / stratification_val.shape[0]
clusters_test_perc = stratification_test['cluster'].sum() / stratification_test.shape[0]

print('The percentage of cluster 1 in the training set:', "{0:.2f}%".format(clusters_training_perc * 100))
print('The percentage of cluster 1 in the validation set:', "{0:.2f}%".format(clusters_validation_perc * 100))
print('The percentage of cluster 1 in the test set:', "{0:.2f}%".format(clusters_test_perc * 100))

print('The distribution of the labels in the training set:', "{0:.2f}%".format(Y_train[Y_train == 1].count() / Y_train.shape[0] * 100),',',
                                                             "{0:.2f}%".format(Y_train[Y_train == 2].count() / Y_train.shape[0] * 100),',',
                                                             "{0:.2f}%".format(Y_train[Y_train == 3].count() / Y_train.shape[0] * 100),',',
                                                             "{0:.2f}%".format(Y_train[Y_train == 4].count() / Y_train.shape[0] * 100),',',
                                                             "{0:.2f}%".format(Y_train[Y_train == 5].count() / Y_train.shape[0] * 100))

print('The distribution of the labels in the validation set:', "{0:.2f}%".format(Y_val[Y_val == 1].count() / Y_val.shape[0] * 100),',',
                                                               "{0:.2f}%".format(Y_val[Y_val == 2].count() / Y_val.shape[0] * 100),',',
                                                               "{0:.2f}%".format(Y_val[Y_val == 3].count() / Y_val.shape[0] * 100),',',
                                                               "{0:.2f}%".format(Y_val[Y_val == 4].count() / Y_val.shape[0] * 100),',',
                                                               "{0:.2f}%".format(Y_val[Y_val == 5].count() / Y_val.shape[0] * 100))

print('The distribution of the labels in the test set:', "{0:.2f}%".format(Y_test[Y_test == 1].count() / Y_test.shape[0] * 100),',',
                                                         "{0:.2f}%".format(Y_test[Y_test == 2].count() / Y_test.shape[0] * 100),',',
                                                         "{0:.2f}%".format(Y_test[Y_test == 3].count() / Y_test.shape[0] * 100),',',
                                                         "{0:.2f}%".format(Y_test[Y_test == 4].count() / Y_test.shape[0] * 100),',',
                                                         "{0:.2f}%".format(Y_test[Y_test == 5].count() / Y_test.shape[0] * 100))

In [ ]:
# write the image_ids in the training, validation and test set to file
image_ids_train.to_frame().reset_index().drop('index', axis = 1).to_csv('results/image_ids_train.csv', sep = ',', index = False)
image_ids_val.to_frame().reset_index().drop('index', axis = 1).to_csv('results/image_ids_val.csv', sep = ',', index = False)
image_ids_test.to_frame().reset_index().drop('index', axis = 1).to_csv('results/image_ids_test.csv', sep = ',', index = False)

In [ ]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

# let's check the importance of the features and select only the relevant ones

clf = ExtraTreesClassifier(random_state = 0).fit(X_train, Y_train)

features = []

for feature, importance in zip(X_train.columns, clf.feature_importances_):
    features.append((importance, feature))

features.sort(reverse = True)
features_relevant = [x[1] for x in features if x[0] > 0.001]

features = pd.DataFrame()

features['feature'] = X_train.columns
features['importance'] = clf.feature_importances_

features.sort_values(by = ['importance'], ascending = True, inplace = True)

# select the 15 most important features for visualization purposes
features = features.iloc[-15:, :]

features.set_index('feature', inplace = True)
features.plot(kind = 'barh',
              figsize = (10, 12),
              color = "#FF9933",
              ec = "black",
              linewidth = 0.5,
              legend = None)

plt.title('\nWhich features are important?\n', fontsize = 14)
plt.xlabel('Importance', fontsize = 11,  labelpad = 10)
plt.ylabel('')

plt.show()

In [ ]:
pd.options.display.float_format = "{0:.3f}".format
print(features.sort_values('importance', ascending = False))

In [ ]:
len(features_relevant)

In [ ]:
# let's scale the data (mean of zero and standard deviation of one) as input to a logistic regression for
# maximum interpretability of the results and perform a sanity check to see whether each feature has in fact
# a mean of zero and standard deviation one

X_train_selection = X_train[features_relevant]
X_val_selection = X_val[features_relevant]
X_test_selection = X_test[features_relevant]

scaler = StandardScaler().fit(X_train_selection)

X_train_selection_scaled = scaler.transform(X_train_selection)
X_val_selection_scaled = scaler.transform(X_val_selection)
X_test_selection_scaled = scaler.transform(X_test_selection)

print('Information about the training set:\n')
print('   - datatype:', X_train_selection_scaled.dtype)
print('   - shape of the dataset:', X_train_selection_scaled.shape)
print('   - sum of the means of the columns:', round(X_train_selection_scaled.mean(axis = 0).sum(), 2))
print('   - sum of the standard deviations of the columns:', round(X_train_selection_scaled.std(axis = 0).sum(), 2))
print('\n')
print('Information about the validation set:\n')
print('   - datatype:', X_val_selection_scaled.dtype)
print('   - shape of the dataset:', X_val_selection_scaled.shape)
print('   - sum of the means of the columns:', round(X_val_selection_scaled.mean(axis = 0).sum(), 2))
print('   - sum of the standard deviations of the columns:', round(X_val_selection_scaled.std(axis = 0).sum(), 2))
print('\n')
print('Information about the test set:\n')
print('   - datatype:', X_test_selection_scaled.dtype)
print('   - shape of the dataset:', X_test_selection_scaled.shape)
print('   - sum of the means of the columns:', round(X_test_selection_scaled.mean(axis = 0).sum(), 2))
print('   - sum of the standard deviations of the columns:', round(X_test_selection_scaled.std(axis = 0).sum(), 2))

In [ ]:
# train a logistic regression classifier and output the accuracy for both the training and validation set

clf = LogisticRegression(multi_class = 'multinomial', solver = 'newton-cg', random_state = 0).fit(X_train_selection_scaled, Y_train)

# accuracy for the training set
Y_train_pred = clf.predict(X_train_selection_scaled)
accuracy_train = accuracy_score(Y_train, Y_train_pred)

# accuracy for the validation set
Y_val_pred = clf.predict(X_val_selection_scaled)
accuracy_validation = accuracy_score(Y_val, Y_val_pred)

print('Accuracy on the training set:', "{0:.2f}".format(accuracy_train))
print('Accuracy on the validation set:', "{0:.2f}".format(accuracy_validation))

In [ ]:
# let's check for multicollinearity

c = X_train_selection.corr().abs()
c *= np.tri(*c.shape)

s = c.unstack()

so = s.sort_values(kind = 'quicksort', ascending = False)
so = so[so < 1]
so.nlargest(50)

# remove the following features due to high correlation with other features:
# - male_18_29_state
# - female_18_29_state
# - state_California
# - temp_min_celsius_1 t/m temp_min_celsius_12
# - temp_max_celsius_1 t/m temp_min_celsius_12

In [ ]:
features_multicollinearity = ['male_18_29_state',
                              'female_18_29_state',
                              'ranking_overall',
                              'temp_max_celsius_min1',
                              'temp_max_celsius_plus1',
                              'temp_min_celsius_min1',
                              'temp_min_celsius_plus1',              
                              'precipitation_mm_min1',
                              'precipitation_mm_min0',
                              'precipitation_mm_plus1',
                              'average_yearly_snowfall',
                              'state_California',
                              'state_Wyoming',
                              
                              'temp_min_celsius_1',
                              'temp_min_celsius_2',
                              'temp_min_celsius_3',
                              'temp_min_celsius_4',
                              'temp_min_celsius_5',
                              'temp_min_celsius_6',
                              'temp_min_celsius_7',
                              'temp_min_celsius_8',
                              'temp_min_celsius_9',
                              'temp_min_celsius_10',
                              'temp_min_celsius_11',
                              'temp_min_celsius_12',
                                 
                              'temp_max_celsius_1',
                              'temp_max_celsius_2',
                              'temp_max_celsius_3',
                              'temp_max_celsius_4',
                              'temp_max_celsius_5',
                              'temp_max_celsius_6',
                              'temp_max_celsius_7',
                              'temp_max_celsius_8',
                              'temp_max_celsius_9',
                              'temp_max_celsius_10',
                              'temp_max_celsius_11',
                              'temp_max_celsius_12',
                             
                              'precipitation_mm_1',
                              'precipitation_mm_2',
                              'precipitation_mm_3',
                              'precipitation_mm_4',
                              'precipitation_mm_5',
                              'precipitation_mm_6',
                              'precipitation_mm_7',
                              'precipitation_mm_8',
                              'precipitation_mm_9',
                              'precipitation_mm_10',
                              'precipitation_mm_11',
                              'precipitation_mm_12']

features_relevant_update = [x for x in features_relevant if x not in features_multicollinearity]

In [ ]:
len(features_relevant), len(features_multicollinearity), len(features_relevant_update)

In [ ]:
# save the relevant features

features_df = pd.DataFrame(features_relevant_update)
features_df.rename(columns = {0: 'feature'}, inplace = True)
features_df.to_csv('results/features_relevant_logistic_regression.csv', sep = ',', index = False)

In [ ]:
# let's scale the data (mean of zero and standard deviation of one) as input to a logistic regression for
# maximum interpretability of the results and perform a sanity check to see whether each feature has in fact
# a mean of zero and standard deviation one

X_train_selection = X_train[features_relevant_update]
X_val_selection = X_val[features_relevant_update]
X_test_selection = X_test[features_relevant_update]

scaler = StandardScaler().fit(X_train_selection)

X_train_selection_scaled = scaler.transform(X_train_selection)
X_val_selection_scaled = scaler.transform(X_val_selection)
X_test_selection_scaled = scaler.transform(X_test_selection)

print('Information about the training set:\n')
print('   - datatype:', X_train_selection_scaled.dtype)
print('   - shape of the dataset:', X_train_selection_scaled.shape)
print('   - sum of the means of the columns:', round(X_train_selection_scaled.mean(axis = 0).sum(), 2))
print('   - sum of the standard deviations of the columns:', round(X_train_selection_scaled.std(axis = 0).sum(), 2))
print('\n')
print('Information about the validation set:\n')
print('   - datatype:', X_val_selection_scaled.dtype)
print('   - shape of the dataset:', X_val_selection_scaled.shape)
print('   - sum of the means of the columns:', round(X_val_selection_scaled.mean(axis = 0).sum(), 2))
print('   - sum of the standard deviations of the columns:', round(X_val_selection_scaled.std(axis = 0).sum(), 2))
print('\n')
print('Information about the test set:\n')
print('   - datatype:', X_test_selection_scaled.dtype)
print('   - shape of the dataset:', X_test_selection_scaled.shape)
print('   - sum of the means of the columns:', round(X_test_selection_scaled.mean(axis = 0).sum(), 2))
print('   - sum of the standard deviations of the columns:', round(X_test_selection_scaled.std(axis = 0).sum(), 2))

In [ ]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

# let's check the importance of the features and select only the relevant ones

clf = ExtraTreesClassifier(random_state = 0).fit(X_train[features_relevant_update], Y_train)

features = []

for feature, importance in zip(X_train[features_relevant_update].columns, clf.feature_importances_):
    features.append((importance, feature))

features.sort(reverse = True)
features_relevant = [x[1] for x in features if x[0] > 0.001]

features = pd.DataFrame()

features['feature'] = X_train[features_relevant_update].columns
features['importance'] = clf.feature_importances_

features.sort_values(by = ['importance'], ascending = True, inplace = True)

# select the 15 most important features for visualization purposes
features = features.iloc[-15:, :]

features.set_index('feature', inplace = True)
features.plot(kind = 'barh',
              figsize = (10, 12),
              color = "#FF9933",
              ec = "black",
              linewidth = 0.5,
              legend = None)

plt.title('\nWhich features are important?\n', fontsize = 14)
plt.xlabel('Importance', fontsize = 11,  labelpad = 10)
plt.ylabel('')

plt.show()

In [ ]:
# train a logistic regression classifier and output the accuracy for both the training and validation set

clf = LogisticRegression(multi_class = 'multinomial', solver = 'newton-cg', random_state = 0).fit(X_train_selection_scaled, Y_train)

# accuracy for the training set
Y_train_pred = clf.predict(X_train_selection_scaled)
accuracy_train = accuracy_score(Y_train, Y_train_pred)

# accuracy for the validation set
Y_val_pred = clf.predict(X_val_selection_scaled)
accuracy_validation = accuracy_score(Y_val, Y_val_pred)

print('Accuracy on the training set:', "{0:.2f}".format(accuracy_train))
print('Accuracy on the validation set:', "{0:.2f}".format(accuracy_validation))

In [ ]:
# accuracy for the test set
Y_test_pred = clf.predict(X_test_selection_scaled)
accuracy_test = accuracy_score(Y_test, Y_test_pred)

print('Accuracy on the test set:', "{0:.2f}".format(accuracy_test))

In [ ]:
coefficients = pd.DataFrame(np.column_stack((X_train_selection.columns.values, clf.coef_.T)))
coefficients.columns = ['feature', '1 - HH', '2 - H', '3 - M', '4 - L', '5 - LL']
coefficients.to_csv('results/LR_baseline_weights.csv', sep = ',', index = False)

In [ ]:
pd.DataFrame(clf.intercept_).to_csv('results/LR_baseline_intercept.csv', sep = ',', index = False)

In [ ]:
pd.DataFrame(X_test_selection_scaled).to_csv('results/LR_baseline_X_test.csv', sep = ',', index = False)

In [ ]:
# let's construct the confusion matrix
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(Y_test, Y_test_pred, labels = None, sample_weight = None)
conf_matrix

In [ ]:
# let's add the predictions to the features to analyse a bit more

predictions = []

for image_id, pred in zip(image_ids_test, Y_test_pred):
    predictions.append((image_id, pred))

predictions_df = pd.DataFrame(predictions)
predictions_df.columns = ['image_id', 'prediction']

In [ ]:
# add the other features and select the relevant ones for this purpose
predictions_df = account_data_02.merge(predictions_df, on = 'image_id', how = 'inner')
predictions_df = predictions_df[['image_id', 'resort', 'cluster', 'likes_groups', 'prediction']]
predictions_df['likes_groups'] = predictions_df['likes_groups'].apply(lambda x: int(str(x)[0]))

In [ ]:
# create an overview of the predictions versus the original categories per resort
overview = predictions_df.groupby(['resort', 'likes_groups', 'prediction']).image_id.count().reset_index().rename(columns = {'image_id': 'count'})
overview['correct'] = np.where(overview['prediction'] == overview['likes_groups'], 1, 0)

In [ ]:
sum_per_group = overview[['resort', 'correct', 'count']].groupby(['resort', 'correct']).sum().reset_index().rename(columns = {'count': 'count_per_group'})
sum_total = overview[['resort', 'count']].groupby(['resort']).sum().reset_index().rename(columns = {'count': 'total'})
percentage = sum_per_group.merge(sum_total, on = 'resort', how = 'outer')
percentage['percentage'] = 100 * percentage['count_per_group'] / percentage['total']

In [ ]:
percentage_top10 = percentage[(percentage['total'] >= 100) & (percentage['correct'] == 1)].nlargest(10, 'percentage') \
                    .sort_values(by = 'percentage', ascending = False).set_index('resort')
percentage_top10

In [ ]:
percentage_worst10 = percentage[percentage['correct'] == 1].nsmallest(10, 'percentage') \
                        .sort_values(by = 'percentage', ascending = False).set_index('resort')
percentage_worst10

In [ ]:
# save files so they can be used in graph of final model
percentage_top10.to_csv('results/LR_baseline_top10.csv', sep = ',', index = False)
percentage_worst10.to_csv('results/LR_baseline_worst10.csv', sep = ',', index = False)

In [ ]:
top10_resorts = percentage_top10.index.values.tolist()
worst10_resorts = percentage_worst10.index.values.tolist()
xticks_labels = [None] * (len(top10_resorts) + len(worst10_resorts))
xticks_labels[::2] = top10_resorts
xticks_labels[1::2] = worst10_resorts

In [ ]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

fig = plt.figure(figsize = (12, 6))
ax = fig.add_subplot(111)

index = np.linspace(0, 19, 10)
bar_width = 0.6

top10 = plt.bar(index,
                percentage_top10['percentage'],
                bar_width,
                color = '#0099FF',
                ec = 'black',
                linewidth = 0.5)

X = np.linspace(0, 19, 10)
Y = np.array(percentage_top10['percentage'])
Z1 = np.array(percentage_top10['count_per_group'])
Z2 = np.array(percentage_top10['total'])

for a, b, c, d in zip(X, Y, Z1, Z2): 
    plt.text(a + 0.05, b + 3, '[' + str(c) + ' / ' + str(d) + ']', fontsize = 8, ha = 'center', va = 'center')
    
worst10 = plt.bar(index + 1,
                  percentage_worst10['percentage'],
                  bar_width,
                  color = '#FF9933',
                  ec = 'black',
                  linewidth = 0.5)

X = np.linspace(1, 20, 10)
Y = np.array(percentage_worst10['percentage'])
Z1 = np.array(percentage_worst10['count_per_group'])
Z2 = np.array(percentage_worst10['total'])

for a, b, c, d in zip(X, Y, Z1, Z2): 
    plt.text(a + 0.05, b + 3, '[' + str(c) + ' / ' + str(d) + ']', fontsize = 8, ha = 'center', va = 'center')

xticks_major = xticks_major = np.linspace(0, 20, 20)
ax.set_xticks(xticks_major)
ax.set_xticklabels(xticks_labels, fontsize = 10, rotation = 'vertical')

plt.ylim(0, 110)
yticks_major = np.round(np.linspace(0, 100, 11), 10)
yticks_major_str = (yticks_major).astype(int).astype(str).tolist()
yticks_labels = [x + ' %' for x in yticks_major_str]
ax.set_yticks(yticks_major)
ax.set_yticklabels(yticks_labels, fontsize = 10)
ax.yaxis.grid(color = '#333333', alpha = 0.25, zorder = 1)
ax.set_axisbelow(True)

legend = plt.legend([top10, worst10],
                ['ten resorts with the highest accuracy',
                'ten resorts with the lowest accuracy'],
                fontsize = 8,
                loc = 1,
                facecolor = 'white',
                edgecolor = 'black',
                borderaxespad = 1)

plt.suptitle('The ten resorts with the highest / lowest accuracy', fontsize = 14, y = 0.97)
plt.title('(number of correct predictions versus number of samples in the test set in brackets)', fontsize = 10, y = 1.02)

plt.xlabel('')
plt.ylabel('Accuracy', fontsize = 11)
plt.show()

filename = 'results/top10_worst10_baseline.png'
fig.savefig(filename)